In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [18]:
ROOT_DIR = '/content/drive/MyDrive/AIChallenges/Zalo22/WikiQA/'

In [2]:
!unzip {ROOT_DIR}/data/e2eqa-train+public_test-v1

Archive:  /content/drive/MyDrive/AIChallenges/Zalo22/WikiQA/data/e2eqa-train+public_test-v1.zip
   creating: e2eqa-train+public_test-v1/
  inflating: e2eqa-train+public_test-v1/zac2022_testa_sample_submission.json  
  inflating: e2eqa-train+public_test-v1/zac2022_testa_only_question.json  
  inflating: e2eqa-train+public_test-v1/zac2022_train_merged_final.json  


In [83]:
import re
import json
import pandas as pd

In [84]:
with open('e2eqa-train+public_test-v1/zac2022_train_merged_final.json', 'r') as f:
    raw_gt = json.load(f)['data']

In [85]:
with open(f'{ROOT_DIR}/valid_predictions/baseline.json', 'r') as f:
    predictions = json.load(f)

In [86]:
for doc in predictions:
    if doc['predicted_question_type'] == 'entity':
        doc['predicted_answer'] = 'wiki/' + re.sub('\\s+', '_', doc['predicted_answer'])

In [87]:
groundtruths = []
for doc in raw_gt:
    if doc['category'] != 'FULL_ANNOTATION':
        continue
    # doc['question_type'] = get_category(doc['answer'])
    groundtruths.append(doc)

In [88]:
len(groundtruths)

4989

In [89]:
gt_df = pd.DataFrame(groundtruths).drop_duplicates()

In [90]:
pred_df = pd.DataFrame(predictions).drop_duplicates(subset=['id'])
pred_df = pred_df[['id', 'predicted_question_type', 'retrieved', 'predicted_answer']]

In [91]:
df = gt_df.merge(pred_df, on='id')

In [92]:
len(df)

4988

In [93]:
len(pred_df)

4988

In [94]:
len(gt_df)

4988

In [95]:
def retrieved_successfully(row):
    short_candidate = row['short_candidate']
    doc = row['retrieved']
    return short_candidate in doc

def answered_correctly(row):
    return row['answer'] == row['predicted_answer']

In [96]:
df['retrieved_correctly'] = df.apply(retrieved_successfully, axis=1)
df['answered_correctly'] = df.apply(answered_correctly, axis=1)

In [97]:
df['retrieved_correctly'].value_counts(normalize=True)

True     0.529872
False    0.470128
Name: retrieved_correctly, dtype: float64

In [98]:
df['answered_correctly'].value_counts(normalize=True)

False    0.788292
True     0.211708
Name: answered_correctly, dtype: float64

In [99]:
cond = (~df['answered_correctly']) & df['retrieved_correctly']

In [101]:
df[cond].sample(10)

,id,question,title,text,short_candidate_start,short_candidate,answer,category,is_long_answer,predicted_question_type,retrieved,predicted_answer,retrieved_correctly,answered_correctly
920,dbae5a550416f31efb65c2f6da880ed0,Phía Bắc của tỉnh Bình Thuận giáp với tỉnh nào,Bình Thuận,Vị trí địa lý . Phía Bắc của tỉnh Bình Thuận g...,54,tỉnh Lâm Đồng,wiki/Lâm_Đồng,FULL_ANNOTATION,True,entity,Bình Thuận\n\nBình Thuận là một tỉnh thuộc vùn...,wiki/Bình_Thuận,True,False
797,b2d91ab21661ae9978a113e1dc343275,Hồ nằm giữa trung tâm thành phố Đà Lạt mang tê...,Hồ Xuân Hương (Đà Lạt),Hồ Xuân Hương là một hồ nước đẹp nằm giữa trun...,0,Hồ Xuân Hương,wiki/Hồ_Xuân_Hương_(Đà_Lạt),FULL_ANNOTATION,True,entity,Hồ Xuân Hương (Đà Lạt)\n\nHồ Xuân Hương là một...,wiki/Hồ_Xuân_Hương,True,False
721,58f469a20c7238207735216b71192b37,tháp eiffel toạ lạc ở công viên nào tại thành ...,Tháp Eiffel,Tháp Eiffel là một công trình kiến trúc bằng t...,59,công viên Champ-de-Mars,wiki/Champ-de-Mars,FULL_ANNOTATION,True,entity,Tháp Eiffel\n\nTháp Eiffel (tiếng Pháp: Tour E...,wiki/ChampdeMars,True,False
845,34aced758513a72b25ad37cdfb90657f,Sau khi Ngô Quyền mất thì ông ủy thác con trai...,Ngô Xương Ngập,"Ông vua mất ngôi . Năm 944 , Ngô Quyền mất , u...",72,Dương Tam Kha,wiki/Dương_Tam_Kha,FULL_ANNOTATION,True,entity,Ngô Nhật Khánh\n\nNgô Nhật Khánh (chữ Hán: 吳日慶...,wiki/Đinh_Toàn,True,False
1300,5d247c21bf7fc7681ae32350b5c3c746,Ai là người sai Nguyễn Trãi viết Bình Ngô đại cáo,Bình Ngô đại cáo,Bình Ngô đại cáo () là bài cáo viết bằng văn n...,107,Bình Định Vương Lê Lợi,wiki/Lê_Thái_Tổ,FULL_ANNOTATION,True,entity,Bình Ngô đại cáo\n\nBình Ngô đại cáo () là bài...,wiki/Nguyễn_Đăng_Na,True,False
3192,2f00753a87865058952ab4b8b511c488,trần quý khoáng là cháu nội của vua,Trùng Quang Đế,Ông có tên huý là Trần Quý Khoáng hay Trần Quý...,72,Trần Nghệ Tông,wiki/Trần_Nghệ_Tông,FULL_ANNOTATION,True,entity,Trùng Quang Đế\n\nTrùng Quang Đế (chữ Hán: 重光帝...,wiki/Trần_Quý_Khoáng,True,False
3309,2fa1f8b351d220cf39f56335cbe98c9b,tàu sân bay duy nhất còn hoạt động của hải quâ...,Đô đốc Kuznetsov ( tàu sân bay Nga ),Tàu sân bay Đô đốc Kuznetsov ( tiếng Nga : Адм...,0,Tàu sân bay Đô đốc Kuznetsov,wiki/Đô_đốc_Kuznetsov_(tàu_sân_bay_Nga),FULL_ANNOTATION,True,entity,Đô đốc Kuznetsov (tàu sân bay Nga)\n\nTàu sân ...,wiki/Kuznetsov,True,False
3514,a3f615ee0ffbd17b57b63806d2832896,hang động tự nhiên lớn nhất thế giới nằm ở tỉn...,Quảng Bình,Du lịch . Hang Sơn Đoòng Là hang động tự nhiên...,209,tỉnh Quảng Bình,wiki/Quảng_Bình,FULL_ANNOTATION,True,entity,Hang Sơn Đoòng\n\nHang Sơn Đoòng là một hang đ...,wiki/Sơn_Đoòng,True,False
4604,62a8de743ba5ea171f388ffae58f251e,đền Đươi được xây dựng ở tỉnh nào,Đền Đươi,"Đền Đươi tạo lạc tại thôn Cẩm Cầu , xã Thống N...",68,tỉnh Hải Dương,wiki/Hải_Dương,FULL_ANNOTATION,True,entity,"Đền Đươi\n\nĐền Đươi tạo lạc tại thôn Cẩm Cầu,...",wiki/Cẩm_Đới,True,False
4669,65ae8a77bf4faaae1fd0bc64be38aafc,Ngày bao nhiêu sẽ đưa ông táo về trời,Lễ cúng ông táo,"chầu Trời , nếu để sang ngày 23 tháng Chạp mới...",24,ngày 23 tháng Chạp,ngày 23 tháng Chạp,FULL_ANNOTATION,True,date,"Lễ cúng ông táo\n\nHàng năm, theo quan niệm củ...",23 tháng Chạp,True,False
